In [53]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import string
import csv
import requests
import json
import time

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

def save_csv(filename,mylist):
  with open(filename, "w") as output:
    writer = csv.writer(output, delimiter=',',lineterminator='\n')
    for val in mylist:
        writer.writerow([val.encode('utf-8')])   

def open_csv(filename):
  mylist = []
  with open(filename, "r") as output:
    spamreader = csv.reader(output, delimiter=',',lineterminator='\n')
    for row in spamreader:
      mylist.append(unicode(row[0], 'utf-8'))
  return mylist

In [42]:
with open('company_list.json') as f:
    startups = json.load(f)

In [59]:
def get_linkedin_founders(user_link):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    }
    page = requests.get(user_link,headers=headers)
    print(page.content)
    html = BeautifulSoup(page.content, 'html.parser')
    founder_angelist_links = []
    for elem in html.select('.founders .section'):
        for photo in elem.select('.photo'):
            founder_angelist_links.append(photo.select('a')[0]['href'])
    founder_linkedin_links = []
    for link in founder_angelist_links:
        time.sleep(1)
        page = requests.get(link,headers=headers)
        html = BeautifulSoup(page.content, 'html.parser')
        for elem in html.select('.fontello-linkedin'):
            founder_linkedin_links.append(elem['href'])
    return founder_angelist_links, founder_linkedin_links

In [60]:
count = 0
for startup in startups[:50]:
    print(str(count) + ' ' + startup['title'])
    time.sleep(1)
    founder_angelist_links, founder_linkedin_links = get_linkedin_founders(startup['link'])
    startup['founders'] = {}
    startup['founders']['angelist'] = founder_angelist_links
    startup['founders']['linkedin'] = founder_linkedin_links
    print(startup)
    count += 1

0 InvestLab
b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="THpO9iR/vCcMh0lmNMdEPN4mIlPtNlX5YqyXH69s39N8ORDV058DdoXHcD4mDDHBivc909uBqXC+o3LebEFgHg==" />\n<!--\nm6ZJVkWWA4PRliLmJUWfUVpjJYR9TluFRrSguDcWj/XVXoONzHS/I/AdKeGhkuP3iP1lvjTI10Kh8iugwJAmiNQXio8rypSUVpujk0WMDusBPiAkbseGwAFFencYcCzRowOW+r+NAeEEFcvmSzsjfcCB7eC2gO7VwvucOXX9XaZNK829pFZNgwox5jofq+lvTAHw5c7ECA+4nTC6UTCUlrT2CJiAYACuwhlRP85LgOHRRfsXPx1JkFh9cDSZL8fva4ucZX1qWha/jOPgceppr6HrBrr+ceG+sr8gnNHh+bxcLQDtEKGUSf5fiXq4rPTNrcgnYoB/Qtn0UM0REdZmegaHVpRVqR86+wDwmRPEh6dAx/YT5b1jSpsORFvI4HPx4TM1oxhr5QKDbyneAE4iIo6TYlL4w9u0n4eUadQxnytpbGNVShe3AbE3nOs3VG60RuZEuqGc1dwpmqQG0Sg/GZS1R5HdRfF0zyFUweGGGOdANFMOledzjSWshCfz7RmjYdW3MoO0KsvsEHyU9NVNy9FWoYV5CVSjLuuOfx2fEcHfHD2kN5I8JsMBgpdJZHJn5V9r7rXFkSUYb3SZ1jyW67wp90xbEuAlSnA7Q0AM/OlE

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="a6zHkTNbDQhJZl2etgrxrXNl8kgPGF8+oV5DIMlixtIxcgrZLW6xgNefdlkoVMe8TksucXqTaOJ1rcy1NNvAyQ==" />\n<!--\n2OPLhSrKm3dd4onHRSxDPz78NifnCJ0Ta2USyqOSeSeSi/vcSuBO85Fw//0HNHs36G2y+v6wTcPAzmYlrdEe8saX6S4NHcQ19wgJvyLpruqSF7o3JbqaKntGaAUn1FqgDdyQEcYhWrzun+J4BcQjn22dlMxTz5BpMwYjbMBr/yVfWwq3nrUW9Y/p96E493HLCCZ+fDcg+N0kouyivuEnFRqzlNFeFscrzWbQLlP9BSPzJ6JG0UwtwyiNuULCQ0BCZRVDGlD9JiJ033+y5buCm1vwUUZnVeBocBHBX/jZw/tdwoPebTTsxCja1QR6SyCt1GFWDcKvjoWrLw8z3MZiHqC3ylSdmNEMHRopomv7xQyOvaWvIuvUDkISWXJtJpXNP6Fs6ny8GV7d/tfG8H9cQaa7goTaZeVBI4iH/b45jzBfHZlNGhCDljmVshsHSdfe1B3WodPymy/c7VbeZGbsABp9v7NvGzvWwTQCr/0sf6Euo/njVytdLbUjcwRR6xbQtMSXnfe+81x37OLq9kOpus0nXcn4lmyAhKv+f9Iv4jU=\n-->\n<meta content=\'text/html; charset=UTF-8\' http-equiv=\'Content-Type\

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="TUGlbZ18LMgWhCDlv3xAObr6JeK8v/atlLVHT4Lkr3B7K2kCmyLDm36lUZ9WJhF8NR5r4a7wzPiBmxMfNRAoRw==" />\n<!--\n/rx0HrrJOPn+Hwo4ngcl6Ee20e9evQV8Xw71LKB/xxTLaGhMU3r/u5WCkBdwrSNyz5T3ctx7CiVyjjANZ8QezEB+azIwfce55cQszgGAnp/7jvHMt9XhEefbWUXNjXFHWcE9vsHleDaPqhsRbsY3elGHqeA6pZiOa0wRZcncyTbiG2AQUoO/cDSMd5EdlVMltuMqzktUcnPi5fikYMeTiKFJdDjm2lIXCBVHb37rniZNo/cG8LJolz+zdXAHBSxLCtbQbandoAe3DJjO+H5DqFd2E5a5TGotwM6zRaTv8s2SdQDokJ+vyIA2779iqo5zPqiirLcBlLCvtUYTU2eR9Hv6TbpjPeLslF2mB5usy9rYafrcYZ1724eIR7MCFkcfo2AkAG8c3mGvp2JHu6SFygQDzqcGjc91uSMpvg0n7qng5ARK/r93VdESMsgV/jy/+FiLD3to3tp2r4Zx0SvZz0hMt9d63mDRrCtfNTb6uIJZ7o9ldg7NyQjTAb0X2FumMHQZrvAUH6KT49LtLBUVT4gs4tug3145m4DWOkmKTQM2bsjRMLrD3Vgw0BWWj3PTg/Z0OZnqFIn4mPOqjXS+Mw3fVc+6Id4VAfAvWNKqNmX8//yHGDT/DAma

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="uAerX9omsqyh0gZ1v3NAUx1LBscOrGKl7RJImXUeby2MaizukuTb8PlPNZd46sUqAGHt9kMZJxZgkyJx44IwhA==" />\n<!--\nHaeGn0n6UIANt5n+nZexKWSqOaBAIGT4WrhFjpUGc3r8G7/PGrZkPAfjSH5zoal2zfgL/ggjpgT5y30bhIdJqGVK1N+ZzBDnX7kE7RWvhyZP89wooPxRjs26Bqwfxozgscz/j8rFZ09W57/wcG7LcGxZ24IlTHi4ah9QjLUge85aFbNxz0jRFqce//bZ/UobK/JfBAH9ta9muWGKQpNIOJieStM6pj4tNIVDLOwwXu530iRRhJepxhyThW8lv7DkCqfX7PyBs5n/GLMm0tOe7QkZtumKLfDw5qnjfGSdK3YeIL37W3vXiOcO06Yrxx9nFQIVJNgzZY6gratsawkDBUivaZZsS7P10Z/9O8AFU9Xmfuu7Ew4NmQ/0Q23t5rw3k4kGI6VCX6OkoU5QGG3X3I6HFcIdJGLMR/hasflMWfPRxn1jGIdv8Ls5TGg9rScbLeawPD82LghqhbWWYMBxM2CsvIuphkUhBIjZzvhgi/7izwur0fkVWLK9IXz269BNWgYIkYlki3cQRMRPI+VFtHQGJA6VAv5B9Iop81Z+GV/hw+M7TZCYBIX803pJwN6iV4bA04ZFKq0aw+lWtzwC43wGtnpn8eadwTo34GFHun9i7puZMCuRUnZZ

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="Y5yTLz8Rn+ge8NmzrclJldwMo4XFw38Njw8H/JsOs3h+EINkRPXu2nE7HweaySG3W4BzgXXCzzwPRzayL/dWYA==" />\n<!--\n83aLTAalIM4QIuT8vwu/oNLkENQ1VgIAHsV7uhUFKsi8LHBCxUk2Qet0fM5TCvueKWnenCGwRsMsKs+Vk4lciSLQZq1tKgo1b8JEHr+5rg/y2b8wPTKfbUIMm+FefLtJ147VE6IeyKPDnaEHH147FjZenN5RKidjPRu3BlIvRtZVXsSchMM94r26UeS4OmcDwXpCwigxsLanAazt07pOizqPRk5RU/l+CLoEBA3E8OSU/3ISyfLQUpfslaCHLa9gV4aHhRQw3igKu/isL0ftzhzP7SjQEQCkbq04aZOT38mlR8qprPHKVbF+rNmmmvpEs6896cRYBh0Fw9K5wyQ7/F+7tdYE8lEyTyEapNnLVGWYLqoru3Ch3mK19h1xMwnl7qpeMic7ZFsUtSXUT1mm1vvhdCyH+GjZRZFJjXyimqpMMezHN/czOE5zfq/s9O2qfig0cTK8F/ROq1EaQ9tK3UZEGzgdX1P0+MTqh1MQCqNceqngyXruclMltLN4H0kvnyh8phmkaeZqMKH1vi2ljqWAwhZ7AhYIB21lq5fFardneGAHQBGgatR954iC2ihyxWUOlto+d/DaLqEwyIrcUwrCe0KYVOzLHRCPbWLvo2NQAT8UcJS6Mm4X

...

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="V0DXXqYnECQPLpwTf16Yp84hkFBP8BwywmmIMErOzTqZw63wBaOIRHGtT+735Q6E5QgOgRFJju1iSSp7qj0BxQ==" />\n<!--\n4qWT+LfZiEv/GeTh6VXszVN2ApkdcfarFRka66JGm+e4ocbl0/GxUi5OJHs8GtFdUNkwFq+esZ9W0hj1rE05poNqj3VCzAdsIvTEdoqMa4bcFcz4DdfKUDfIPGHGmjq0hYCFrJzR36DsPHQcJZI/zOhsRX0hcs0B91NmvpDv6+5Lh9+f2wnjeCHZas40jHz9JILj8HnlYqmSNf9JSBO6jLFzjtd4MePDRGOH+HIyYCQFBzSGFVousCS/A1SUhktc9w0XzVtw6HL/qvsbLZyt43cAvMAJ86hYyLbLU4d3PUtV2O0Y0efe4Y2ON/5vXJCiDY6Jh2mRvhQwg2kGtFfD54hApr/FJfkNAsCLDAfWXhyEZA+fYijsUYpII75FaX6PLBQkVDEtv0oo+UnBXIxCXMucAmxohiXjXFLSi+3pQdMnDV44HKf1rPp9i+Bup51wP9DuH6Xe6Yq2LZ0EgskzVOF77/VW+u/odYn7oaODVRnl0AIleU7boFi9reQfkFK7r++q95hff/6PqHrxlG4QWGdrbBtgQGLdH5xZiHMyYoip+VmPspdBvHP4mCKU05F4Onn8cbXaKnei7u/QqjqdmB75mzuEinoaTVATIuIpllVCgRYAdkF0Bi0c

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="FkpZJfghhK96lb+/H8ohGbbBBNYFeGQS62Ua434HJo7lPZt+lIpCa94IZxib5/UOK7fGGYsW2w7dtyooxDnvZA==" />\n<!--\nRNnqzy4QlPuouJlGP/v5pl4Pdg3jBMwEeCDSZnaPoPA1IcltdPSiZ9hwltTjcyPp2eZ7EyGUVJPoqki7m7iwotUs6PIqRWBZrByAgS0kPa/c4A6Siumc5IvKJG56Eg35KSVoMe01z1w6dWVXh708cXfug2qQfL8XKNJmemN9XDWAM5inHln9RneVMSqsU0vtv8G/t0ugRDMBVhE0Suu7zjetsqS/WwO+isEyDBh2Z5Y6sFooLl7qFXu0Hkb7qUrKCCsbrbJ8spi4ViWoLVC3ow9WUx8U/m1qapn+GKvPHFmgQV6Yb5ZzWMIiG1fz4EUKM6fzpm5Ntqc8+afFsi2nyAkatEAHPaW4Lgm/3+aFN7BPJYyWUTRSHsyChZkanotBVSw83d/JSjEDV0nF4iAS0WvfX4Cvonl/oKZR0f3EPwiBYdzZ0W36xCk/ZXPZcjiOCpZ1QRAXWaqDs+81kM++hD+JxfO5eX2LjmISePXCaQYmW1d4CiUHpg8h4ZooaqgKhgeFTFTDKJmJjUzher8YpbXLlEIOqZUdMkG+dyJxoP32P5zqq0yPMJDwuSE7UN3PrmqxDmX0gDWAHNJIDiTVTt/TnTMe67T5sTRnejXEPQQBHU/BXgRoX3W4

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="oJA398NQu0CKiI6xRGbiZ9hlCi75vYKeQWk9Uw86pIQm+FVZB3osSEXh82iF7o8GP2JESVVNZmvOt7baKi88jw==" />\n<!--\ngpWFvjSE/+t1qVQGvOBnstMLpyvy2+RV+EYZfqJzuEHw8bfO3GxRvkLwCQjVqxUZgDESgZXbDpuJi4U5gKTgDdxwaN8BWBRabwXvRwqumSBSSr1PB6PUZPmfe6z2XsKRaecIRwVgdoJFRalG4ULiQLo1w7JRFCaWzsUw0nmk4AY2OktaX0T9T9Ix6K9tF4/OoUZVcIk9JBF3fho4xNSRmzx1Y7VkpHsAD4SQEky8uC6HC7sRM/edMymv6q3vmIWRzIj5ZLa2BizCE2JJ0Ouz5SN4QHPvfnXacNvvzxlDNkn3Fp4y92hEvpu167suhCOFK2FlbT8BSiPEVqTlw4Mps6FUc1g9gSZbpBugLF3tgoaGHwJUdwIG2yJP72BlEZHOjGBVLUr9sfC8hUcY2UJX2zz5BUYCZDKTJM93HtKiIYdgIX+FdRwcXk1VoV+87d1L+2susZYoMf+MozvkGiuqLlyW4M6d/7KxGoLX9Vw4dE3IIFUawFS8uahF+nrrCp3mpFO8v1MHjLNMF18eQATYFg8GCapR0RHqiF6dOAYHCdIxpY6cs55+eu6FdTnkpn7RpZuhjMNarIJTG45LR62Dm7S1BNpwWLrx7ZbeuEn/+RRxbNyPOoMP1Yo9

b'<!DOCTYPE html>\n<html class=\'notranslate no-js scheme- ml no-ms no-mm size-fixed_width not_responsive\' xmlns:fb=\'http://ogp.me/ns/fb#\'>\n<head>\n<title>AngelList</title>\n<meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="8BmYQcDVkEzwhAaS/3pzOCenl/OgTFCfKH+rC4ICu0emNKB3NzEbPr9wfZvchMlQQyS+TBSESnAP1EvE8NjdDw==" />\n<!--\naL8Y/TVJbozmOqodS5Gmxj8BoROV6MXe0HO3x6UAE3J5ofVhZyu5li5PyUfuFBuC+HgL/PSEvXHBD2Q9ycono5ighDvL1v7ZMXc0/5NtG3Z2lbwxEy79YKIp/1NaUSjKZ9Q3YRROsQECMbwrzjq7uCqu9kpwSs2msMqgaS03P/FNOo0RBusmOVtOd66NUP4rBEZwhj0/y5RgVq4VPwZZE+JbJcKnn13VTnRZ8BFM37mSWZGRwqVQgXE2SDGBAP8JRnH5F/gGn+UmPiSjlvZl8Yn7NsEV+GjaeQVMT7USlV7aNoH2meP8Bjl57EJ4hw1MsRbcre+tLynLzPs8//wQC5WeJyJvaDwYgTPDpVJqtLMqhrvryKfBh8nRuG44V/1VyXvOKsXU+YE3EGKKVJ1yODvYP0o8X/s15G4QO6h4F7fMrSwIAY03uXX786kSdiIpEldaNmWketHBWWg6wb8mliYlGFakzXtg9wRYHYahtghxr03AS1TmSa4TdW9wspZH8LoC5OvgLXtk3PXaW/kwiSn7d0DkIYs7VmZciYpoguUfzkaX4BnFVSM7Xz8ONcRLBL+YimPMeti8KS1dZQxCGR142yhfkcCCsJNKEK4hRelY+8Ecv6a+LBCl

KeyboardInterrupt: 

In [50]:
startups

[{'founders': {'angelist': ['https://angel.co/mark-munoz',
    'https://angel.co/nathan-emerson'],
   'linkedin': ['http://hk.linkedin.com/pub/mark-munoz/0/598/573']},
  'link': 'https://angel.co/investlab',
  'title': 'InvestLab'},
 {'founders': {'angelist': ['https://angel.co/simon-newstead',
    'https://angel.co/kevin-yoon'],
   'linkedin': ['http://hk.linkedin.com/in/simonnewstead']},
  'link': 'https://angel.co/frenzoo',
  'title': 'Frenzoo'},
 {'founders': {'angelist': ['https://angel.co/scott-phillips',
    'https://angel.co/marcos-steverlynck'],
   'linkedin': ['http://www.linkedin.com/in/smphillips']},
  'link': 'https://angel.co/rise-art',
  'title': 'Rise Art'},
 {'founders': {'angelist': [], 'linkedin': []},
  'link': 'https://angel.co/dragon-innovation',
  'title': 'Dragon Innovation'},
 {'founders': {'angelist': [], 'linkedin': []},
  'link': 'https://angel.co/vectr-ventures',
  'title': 'Vectr Ventures'},
 {'founders': {'angelist': [], 'linkedin': []},
  'link': 'https: